In [33]:
import pandas as pd
import numpy as np

elephant_data = pd.read_csv("character+font+images/ELEPHANT.csv")

# display(elephant_data)
pixel_columns = [col for col in elephant_data.columns if col.startswith('r') and 'c' in col]

keep_columns = ['m_label'] + pixel_columns

filtered_elephant_data = elephant_data[keep_columns]

display(filtered_elephant_data)

,m_label,r0c0,r0c1,r0c2,r0c3,r0c4,r0c5,r0c6,r0c7,r0c8,...,r19c10,r19c11,r19c12,r19c13,r19c14,r19c15,r19c16,r19c17,r19c18,r19c19
0,64258,1,1,1,1,1,5,66,151,189,...,57,124,243,255,255,255,255,255,179,105
1,64257,1,1,1,1,1,1,30,123,151,...,57,124,243,255,255,255,255,255,179,105
2,61442,1,1,1,1,1,5,66,151,189,...,57,124,243,255,255,255,255,255,179,105
3,61441,1,1,1,1,1,1,30,123,151,...,57,124,243,255,255,255,255,255,179,105
4,9674,1,1,1,1,1,1,1,110,255,...,255,161,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907,37,1,1,1,30,112,161,161,161,54,...,1,1,50,67,67,32,1,1,1,1
908,36,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
909,35,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
910,34,1,1,1,31,119,255,255,255,255,...,255,153,11,1,1,1,1,1,1,1


In [34]:
# creating dicitionaries for mapping the 

def dictionaries(df):
    #grabs the unique values and sorts them in ascending order
    unique_unicode_v = sorted(df['m_label'].unique())
    #unicode is they key and index is the value in the dictionary
    unicode_to_index = {unicode: index for index, unicode in enumerate(unique_unicode_v)}
    #reverse mapping of the unicode and index
    index_to_unicode = {index: unicode for unicode, index in unicode_to_index.items()}
    return unicode_to_index, index_to_unicode

unicode_to_index, index_to_unicode = dictionaries(elephant_data)


def transform(df, unicode_to_index):

    #normalizing pixel value
    df[pixel_columns] = df[pixel_columns]/256.0

    #extracting the pixel data as a numpy array
    Xs = df[pixel_columns].values.reshape(-1,20,20)

    #transforming unicode label to indices
    Ys = np.array([unicode_to_index[unicode] for unicode in df['m_label'].values])

    return Xs, Ys


X, Y = transform(filtered_elephant_data, unicode_to_index)

print(X.shape)
print(Y.shape)

def print_unicode_mapping(index_to_unicode):
    for index, unicode in index_to_unicode.items():
        print(f"index: {index}, unicode: {unicode} char: {chr(unicode)}")

print_unicode_mapping(index_to_unicode)

(912, 20, 20)
(912,)
index: 0, unicode: 33 char: !
index: 1, unicode: 34 char: "
index: 2, unicode: 35 char: #
index: 3, unicode: 36 char: $
index: 4, unicode: 37 char: %
index: 5, unicode: 38 char: &
index: 6, unicode: 39 char: '
index: 7, unicode: 40 char: (
index: 8, unicode: 41 char: )
index: 9, unicode: 42 char: *
index: 10, unicode: 43 char: +
index: 11, unicode: 44 char: ,
index: 12, unicode: 45 char: -
index: 13, unicode: 46 char: .
index: 14, unicode: 47 char: /
index: 15, unicode: 48 char: 0
index: 16, unicode: 49 char: 1
index: 17, unicode: 50 char: 2
index: 18, unicode: 51 char: 3
index: 19, unicode: 52 char: 4
index: 20, unicode: 53 char: 5
index: 21, unicode: 54 char: 6
index: 22, unicode: 55 char: 7
index: 23, unicode: 56 char: 8
index: 24, unicode: 57 char: 9
index: 25, unicode: 58 char: :
index: 26, unicode: 59 char: ;
index: 27, unicode: 60 char: <
index: 28, unicode: 61 char: =
index: 29, unicode: 62 char: >
index: 30, unicode: 63 char: ?
index: 31, unicode: 64 char:

/var/folders/3w/mkw0gknj67s61glj_9k16q480000gn/T/ipykernel_51537/305023612.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[pixel_columns] = df[pixel_columns]/256.0


In [35]:
import torch
import torchvision

import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataloader import default_collate

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        c1Out = 6
        c2Out = 16

        #defining the first colvolution layer 
        self.conv1 = nn.Conv2d(1, c1Out, 3) #1-d input, c1out outputs, filter size 3x3 pixels
        self.pool = nn.MaxPool2d(2,2) #downsample 2x2 blocks to 1 value

        #define the second convolution layer 
        self.conv2 = nn.Conv2d(c1Out, c2Out, 3) #inputs come from conv1, specify outputs using 3x3 blocks

        #define fully connected layers
        self.fc1 = nn.Linear(c2Out *5 *5 , 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        x = self.conv1(x)
        print(x.shape)
        x = F.relu(x)
        print(x.shape)

        x = self.pool(x)
        print(x.shape)

        #split into 2 line above
        x = self.pool(F.reult(self.conv2(x)))
        print(x.shape)

        x = x.view(-1, self.num_flat_features(x))

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        x = self.fc3(x)

        return x


        


